In [1]:
import mwclient
import time
import transformers

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Tether (cryptocurrency)']

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]

OrderedDict([('revid', 1228660913),
             ('parentid', 1228637363),
             ('minor', ''),
             ('user', 'Schazjmd'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=13, tm_min=22, tm_sec=20, tm_wday=2, tm_yday=164, tm_isdst=-1)),
             ('comment',
              'removed duplicate wikilink ([[m:Special:MyLanguage/User:Jon Harald Søby/diffedit|diffedit]])')])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [5]:
revs[0]

OrderedDict([('revid', 813908017),
             ('parentid', 0),
             ('user', 'Chrisclear'),
             ('timestamp',
              time.struct_time(tm_year=2017, tm_mon=12, tm_mday=5, tm_hour=21, tm_min=55, tm_sec=3, tm_wday=1, tm_yday=339, tm_isdst=-1)),
             ('comment', 'creating article')])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google._upb._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\SUBRAT\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to forc

In [7]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

In [8]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [9]:
import pandas as pd
edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [10]:
edits_df

,edit_count,sentiment,neg_sentiment
2017-12-05,15,0.078047,0.466667
2017-12-07,2,-0.005037,0.500000
2017-12-09,1,0.748121,0.000000
2017-12-11,3,-0.936890,1.000000
2017-12-14,1,0.748121,0.000000
...,...,...,...
2024-05-10,3,0.278935,0.333333
2024-05-14,2,-0.081897,0.500000
2024-05-20,2,-0.905240,1.000000
2024-06-09,4,-0.471948,0.750000


In [11]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits

,edit_count,sentiment,neg_sentiment
2018-02-13,3.166667,-0.238520,0.596111
2018-02-18,2.733333,-0.273916,0.613889
2018-02-21,2.766667,-0.306809,0.630556
2018-02-24,2.833333,-0.301698,0.630556
2018-02-25,2.833333,-0.302900,0.630556
...,...,...,...
2024-05-10,2.066667,-0.236004,0.605556
2024-05-14,2.066667,-0.271010,0.622222
2024-05-20,2.100000,-0.326122,0.655556
2024-06-09,2.200000,-0.309287,0.647222


In [12]:
rolling_edits.to_csv("tether_sentiment.csv")